Working with REST APIs 
CoinGeckoAPI

In [9]:
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()
bitcoin_data = cg.get_coin_market_chart_by_id(id='bitcoin', vs_currency='inr' , days = 30)

In [10]:
bitcoin_data['prices'][:3]

[[1667502117042, 1675285.5757107201],
 [1667505683228, 1674488.4805740379],
 [1667509239541, 1674108.0429320056]]

In [11]:
bitcoin_price_data = bitcoin_data['prices']

bitcoin_price_data[0:5]


[[1667502117042, 1675285.5757107201],
 [1667505683228, 1674488.4805740379],
 [1667509239541, 1674108.0429320056],
 [1667512813230, 1674715.3435067774],
 [1667516492454, 1669291.9000057394]]

Converting the list to a DataFrame

In [12]:
import pandas as pd
data = pd.DataFrame(bitcoin_price_data , columns=["Timestamp" , "Price"])

In [13]:
data.head()

,Timestamp,Price
0,1667502117042,1.675286e+06
1,1667505683228,1.674488e+06
2,1667509239541,1.674108e+06
3,1667512813230,1.674715e+06
4,1667516492454,1.669292e+06


Now that we have the DataFrame we will convert the timestamp to datetime and save it as a column called <code>Date</code>. We will map our <code>unix_to_datetime</code> to each timestamp and convert it to a readable datetime.


In [15]:
import datetime
data['date'] = data['Timestamp'].apply(lambda d: datetime.date.fromtimestamp(d/1000.0))

In [16]:
data.head()

,Timestamp,Price,date
0,1667502117042,1.675286e+06,2022-11-04
1,1667505683228,1.674488e+06,2022-11-04
2,1667509239541,1.674108e+06,2022-11-04
3,1667512813230,1.674715e+06,2022-11-04
4,1667516492454,1.669292e+06,2022-11-04


Using this modified dataset we can now group by the <code>Date</code> and find the min, max, open, and close for the candlesticks.


In [17]:
candlestick_data = data.groupby(data.date, as_index=False).agg(
    {"Price": ['min', 'max', 'first', 'last']})


In [21]:
candlestick_data

date         Price                                          
                         min           max         first          last
0   2022-11-04  1.669292e+06  1.742515e+06  1.675286e+06  1.707075e+06
1   2022-11-05  1.723656e+06  1.755157e+06  1.723656e+06  1.748064e+06
2   2022-11-06  1.733793e+06  1.749073e+06  1.744375e+06  1.739756e+06
3   2022-11-07  1.692764e+06  1.739860e+06  1.739860e+06  1.692764e+06
4   2022-11-08  1.579215e+06  1.707686e+06  1.694246e+06  1.601507e+06
5   2022-11-09  1.386121e+06  1.523173e+06  1.523173e+06  1.386121e+06
6   2022-11-10  1.280554e+06  1.442257e+06  1.337982e+06  1.394824e+06
7   2022-11-11  1.354731e+06  1.440305e+06  1.399444e+06  1.354731e+06
8   2022-11-12  1.338180e+06  1.375740e+06  1.356469e+06  1.361815e+06
9   2022-11-13  1.331424e+06  1.361566e+06  1.358483e+06  1.332396e+06
10  2022-11-14  1.293748e+06  1.379135e+06  1.329584e+06  1.343792e+06
11  2022-11-15  1.320183e+06  1.380079e+06  1.347592e+06  1.376902e+06
12  2022-11-16  1.341004e+06  1.382809e+06  1.354311e+06  1.350233e+06
13  2022-11-17  1.346489e+06  1.361252e+06  1.348925e+06  1.359168e+06
14  2022-11-18  1.354961e+06  1.381672e+06  1.363913e+06  1.354961e+06
15  2022-11-19  1.354220e+06  1.361958e+06  1.354419e+06  1.354989e+06
16  2022-11-20  1.350103e+06  1.365447e+06  1.355766e+06  1.350103e+06
17  2022-11-21  1.311110e+06  1.352327e+06  1.352327e+06  1.312184e+06
18  2022-11-22  1.280231e+06  1.325613e+06  1.307634e+06  1.316622e+06
19  2022-11-23  1.319486e+06  1.360376e+06  1.319486e+06  1.338373e+06
20  2022-11-24  1.346233e+06  1.367583e+06  1.348229e+06  1.357010e+06
21  2022-11-25  1.336925e+06  1.356737e+06  1.354085e+06  1.349115e+06
22  2022-11-26  1.346535e+06  1.359816e+06  1.346535e+06  1.350170e+06
23  2022-11-27  1.341280e+06  1.354204e+06  1.347609e+06  1.350237e+06
24  2022-11-28  1.317958e+06  1.355543e+06  1.351781e+06  1.324376e+06
25  2022-11-29  1.320260e+06  1.350903e+06  1.320260e+06  1.337931e+06
26  2022-11-30  1.340063e+06  1.388663e+06  1.340063e+06  1.365249e+06
27  2022-12-01  1.380161e+06  1.400725e+06  1.381144e+06  1.383570e+06
28  2022-12-02  1.370811e+06  1.388749e+06  1.374939e+06  1.384048e+06
29  2022-12-03  1.379702e+06  1.396100e+06  1.381668e+06  1.383265e+06
30  2022-12-04  1.383718e+06  1.383718e+06  1.383718e+06  1.383718e+06

Plotting

In [19]:
import plotly.graph_objects as go
fig = go.Figure(data=[go.Candlestick(x=candlestick_data['date'],
                open=candlestick_data['Price']['first'],
                high=candlestick_data['Price']['max'],
                low=candlestick_data['Price']['min'],
                close=candlestick_data['Price']['last'])
                      ])

fig.update_layout(xaxis_rangeslider_visible=False)

fig.show()
